In [ ]:
!pip install hub

In [ ]:
# In case you haven't registered before on hub before. Run this cell
!hub register

In [ ]:
!hub login

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
BASE_DIR = '.'
RANDOM_SEED = 7 # for reproducibility

CNN_SAVE_DIR = os.path.join(BASE_DIR, 'models')
os.makedirs(CNN_SAVE_DIR, exist_ok=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "vgg"

# Number of classes in the dataset
num_classes = 3

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for, first 10 will be training the new layers, last 10 the whole model
num_epochs = 20

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = models.vgg11_bn(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
    input_size = 224
    return model_ft, input_size

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
model_ft

In [ ]:
# you can modify the classifier part of the model by doing this
# model_ft.classifier = nn.Sequential(
#     nn.Linear(in_features=25088, out_features=4096, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5),
#     nn.Linear(in_features=4096, out_features=256, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5),
#     nn.Linear(in_features=256, out_features=3, bias=True),
# )

In [ ]:
from hub import Transform, dataset

In [ ]:
data_transforms_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(input_size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_transforms_val=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# these are hub transformations applied to the dataset. Read more here https://docs.activeloop.ai/en/latest/concepts/transform.html
class TrainTransformer(Transform):
    def meta(self):
        return {
            "nightlights_bin": {"shape": (1,), "dtype": "int", "dtag" : "text"},
            "image": {"shape": (1,), "dtype": "object", "chunksize": 100, "dtag" : "image"},
        }

    def forward(self, input):
        ds = {}
        ds["nightlights_bin"] = np.empty(1, dtype="int")
        ds["nightlights_bin"][0] = input["nightlights_bin"]

        ds["image"] = np.empty(1, object)
        ds["image"][0] = data_transforms_train(input["image"])
        return ds

class ValTransformer(Transform):
    def meta(self):
        return {
            "nightlights_bin": {"shape": (1,), "dtype": "int", "dtag" : "text"},
            "image": {"shape": (1,), "dtype": "object", "chunksize": 100, "dtag" : "image"},
        }

    def forward(self, input):
        ds = {}
        ds["nightlights_bin"] = np.empty(1, dtype="int")
        ds["nightlights_bin"][0] = input["nightlights_bin"]

        ds["image"] = np.empty(1, object)
        ds["image"][0] = data_transforms_val(input["image"])
        return ds
    
# converts the dataset from dictionary to pair, as required by the model
def to_pair(input):
    sample=input["image"]
    target=input["nightlights_bin"]
    return (sample,target)

In [ ]:
# Load data
ds = dataset.load("omdena/predicting-poverty-replication-full")

# Splitting back into the original train and test sets
val_ds = ds[0:7393]
train_ds = ds[7393:]

# Applying the tranformation
val_ds = dataset.generate(ValTransformer(),val_ds)
train_ds = dataset.generate(TrainTransformer(),train_ds)

# converting the dataset to pytorch tensors, using to_pair to convert the data from dict to pair
val_ds = val_ds.to_pytorch(to_pair)
train_ds = train_ds.to_pytorch(to_pair)

val_loader = torch.utils.data.DataLoader(val_ds, batch_size=8,shuffle=True)
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=8,shuffle=True)

dataloaders_dict={"train":train_loader,"valid":val_loader}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

In [ ]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=1e-4, momentum=0.9)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        if epoch > 10:
            # fine tune whole model
            for param in model_ft.parameters():
                param.requires_grad = True
            optimizer = optim.SGD(model_ft.parameters(), lr=1e-4, momentum=0.9)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

In [ ]:
path = os.path.join(CNN_SAVE_DIR, 'trained_model.pt')
assert not os.path.isfile(path), print('A model is already saved at this location')
print(f'Saving model to {path}')
torch.save(model_ft, path)

In [ ]:
# you can run below if you want to see the final accuracy on nightlights over the train set
model_ft.eval()   # Set model to evaluate mode

criterion = nn.CrossEntropyLoss()
running_loss = 0.0
running_corrects = 0
total = 0

# Iterate over data.
for inputs, labels in tqdm(dataloaders_dict['train']):
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)

    # statistics
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)

    total += len(preds)
        
print(running_corrects.double()/total)